# 기본과제: DistilBERT로 뉴스 기사 분류 모델 학습하기

In [40]:
!pip install tqdm boto3 requests regex sentencepiece sacremoses datasets

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[MYCDOE] 로그 기능 추가
- 데이터를 확인하기 위해 로그를 통해 데이터를 출력

In [41]:
import torch
from datasets import load_dataset
from torch.utils.data import DataLoader
import logging as log
import matplotlib.pyplot as plt
from collections import Counter
import pandas as pd

log.basicConfig(
    level= log.DEBUG,
)
device = 'mps'
tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'distilbert-base-uncased')

Using cache found in /Users/naseunghoo/.cache/torch/hub/huggingface_pytorch-transformers_main
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /distilbert-base-uncased/resolve/main/tokenizer_config.json HTTP/11" 200 0


In [42]:
log.debug(f"tokenizer={tokenizer}",)

DEBUG:root:tokenizer=DistilBertTokenizerFast(name_or_path='distilbert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}


In [43]:
ds = load_dataset("fancyzhx/ag_news")

DEBUG:urllib3.connectionpool:https://huggingface.co:443 "GET /api/datasets/fancyzhx/ag_news HTTP/11" 200 2832
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): s3.amazonaws.com:443
DEBUG:urllib3.connectionpool:https://s3.amazonaws.com:443 "HEAD /datasets.huggingface.co/datasets/datasets/fancyzhx/ag_news/fancyzhx/ag_news.py HTTP/11" 404 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "GET /api/datasets/fancyzhx/ag_news HTTP/11" 200 2832
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /datasets/fancyzhx/ag_news/resolve/eb185aade064a813bc0b7f42de02595523103ca4/README.md HTTP/11" 200 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /datasets/fancyzhx/ag_news/resolve/eb185aade064a813bc0b7f42de02595523103ca4/.huggingface.yaml HTTP/11" 404 0
DEBUG:urllib3.connectionpool:Startin

[MYCODE] dataset features 확인하기 

In [44]:
log.debug(f"{ds["test"].features['label']}")

DEBUG:root:ClassLabel(names=['World', 'Sports', 'Business', 'Sci/Tech'], id=None)


[MYCODE] AG_News dataset 준비 
- Truncation 삭제 -> False 변경한다.

In [45]:
def collate_fn(batch):
  max_len = 400
  texts, labels = [], []
  for row in batch:
    labels.append(row['label'])
    texts.append(row['text'])
  
  token = tokenizer(texts, padding=True, truncation=False, max_length=max_len,return_tensors='pt')
  texts = torch.LongTensor(token.input_ids)
  labels = torch.LongTensor(labels)
  attention_mask = token.attention_mask
  return texts, attention_mask, labels


train_loader = DataLoader(
    ds['train'], batch_size=64, shuffle=True, collate_fn=collate_fn
)

test_loader = DataLoader(
    ds['test'], batch_size=64, shuffle=False, collate_fn=collate_fn
)

In [46]:
length = []
next(iter(train_loader))
#for idx, text in train_loader:
#    length.append(len(text)) 


#max(length)

(tensor([[  101, 10651,  1019,  ...,     0,     0,     0],
         [  101,  7206, 14300,  ...,     0,     0,     0],
         [  101,  1037, 12185,  ...,     0,     0,     0],
         ...,
         [  101, 19938,  6731,  ...,     0,     0,     0],
         [  101, 11603,  1024,  ...,     0,     0,     0],
         [  101, 10567,  2758,  ...,     0,     0,     0]]),
 tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 tensor([2, 1, 3, 0, 1, 1, 2, 1, 3, 1, 1, 3, 1, 2, 1, 1, 0, 2, 2, 2, 3, 1, 0, 2,
         3, 2, 3, 1, 3, 1, 1, 2, 3, 0, 2, 2, 1, 2, 0, 3, 3, 1, 1, 3, 2, 3, 1, 0,
         2, 0, 3, 1, 1, 0, 3, 1, 1, 2, 1, 1, 0, 1, 3, 0]))

In [47]:
len(train_loader)
log.debug(f"last dataset [text]  word : {train_loader.dataset['text'][119999]}")
log.debug(f"last dataset [label] word : {train_loader.dataset['label'][119999]}")

# 라벨별 빈도 계산
label_counts = Counter(train_loader.dataset['label'])

# DataFrame 생성
label_df = pd.DataFrame(list(label_counts.items()), columns=["Label", "Count"])

# 라벨 이름 추가 (예: AG News 라벨은 0, 1, 2, 3으로 구성됨)
label_names = ["World", "Sports", "Business", "Sci/Tech"]
label_df["Label Name"] = [label_names[label] for label in label_df["Label"]]

print(label_df)

DEBUG:root:last dataset [text]  word : Nets get Carter from Raptors INDIANAPOLIS -- All-Star Vince Carter was traded by the Toronto Raptors to the New Jersey Nets for Alonzo Mourning, Eric Williams, Aaron Williams, and a pair of first-round draft picks yesterday.
DEBUG:root:last dataset [label] word : 1


   Label  Count Label Name
0      2  30000   Business
1      3  30000   Sci/Tech
2      1  30000     Sports
3      0  30000      World


[MYCODE] vocab 데이터 확인

In [48]:
vocab = tokenizer.vocab
vocab_size = len(vocab)
log.debug(f"vocab :{vocab}")
log.debug(f"vocab size : {vocab_size}")

vocab_items = tokenizer.vocab.items();
log.debug(f"vocab item : {vocab_items}")

DEBUG:root:vocab :{'albania': 10407, 'ashton': 13772, 'anxious': 11480, 'drifting': 15013, 'recycling': 17874, 'recreated': 29414, 'operations': 3136, '1976': 3299, 'golf': 5439, 'nesting': 21016, 'fluorescent': 22184, '##oman': 20778, '##space': 23058, 'explains': 7607, 'second': 2117, 'publishers': 8544, '##rgeon': 28242, 'ios': 16380, '人': 1756, 'marjorie': 21562, 'scandals': 29609, 'mor': 22822, '##ots': 12868, 'revelations': 22191, '##→': 30113, 'coffee': 4157, 'deployment': 10813, '##nir': 29339, 'universal': 5415, 'foot': 3329, '##open': 26915, 'conscription': 26329, 'unused': 15171, 'ಾ': 1402, 'bravery': 16534, 'limit': 5787, 'referendum': 9782, 'towels': 24213, 'belgian': 6995, '##dium': 12811, 'bud': 13007, '[unused706]': 711, 'dinner': 4596, '##uda': 14066, 'migrate': 22806, 'fascinating': 17160, '##sari': 22740, '##ivate': 21466, 'tests': 5852, 'aisles': 25442, 'description': 6412, '##ossa': 21842, '[unused881]': 886, 'its': 2049, 'jeff': 5076, '##nn': 10695, '##tted': 1619

[MYCODE] train_dataset 확인하기
- 전체 데이터가 120,000 존재
- 배치사이즈 64로 설정 
    - 120,000 / 64 = 1875
- 토큰별 수 확인하기

In [49]:
log.debug(f"train_loader size = {len(train_loader)}")

token_counts = Counter()

for batch in train_loader:
    input_ids = batch[0]  # 첫 번째 요소가 입력 데이터 (토큰 ID)
    token_counts.update(input_ids.flatten().tolist())  # Counter에 업데이트


token_df = pd.DataFrame(list(token_counts.items()), columns=["token", "Count"])

# 토큰별 개수 출력
print(token_df.head().sort_values('token'))

DEBUG:root:train_loader size = 1875
/opt/anaconda3/envs/myenv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:2847: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


   token   Count
0    101  120000
3   1011  139220
4   1041    2687
2   4095     420
1  17410      10


In [50]:
model = torch.hub.load('huggingface/pytorch-transformers', 'model', 'distilbert-base-uncased')
model

Using cache found in /Users/naseunghoo/.cache/torch/hub/huggingface_pytorch-transformers_main
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /distilbert-base-uncased/resolve/main/config.json HTTP/11" 200 0


DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0-5): 6 x TransformerBlock(
        (attention): MultiHeadSelfAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): Li

In [57]:
from torch import nn

class TextClassifier(nn.Module):
  def __init__(self):
    super().__init__()

    self.encoder = torch.hub.load('huggingface/pytorch-transformers', 'model', 'distilbert-base-uncased')
    self.classifier = nn.Linear(768, vocab_size)

  def forward(self, x, attention_mask):
    outputs = self.encoder(input_ids=x, attention_mask=attention_mask)
    cls_output = outputs['last_hidden_state'][:, 0]  # [CLS] 토큰 벡터 추출

    # 분류 레이어 통과
    logits = self.classifier(cls_output)
    return logits


model = TextClassifier()

Using cache found in /Users/naseunghoo/.cache/torch/hub/huggingface_pytorch-transformers_main
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /distilbert-base-uncased/resolve/main/config.json HTTP/11" 200 0


In [52]:
for param in model.encoder.parameters():
  param.requires_grad = False

In [53]:
import numpy as np
import matplotlib.pyplot as plt
def live_plot(train_accs, test_accs, title="Real-time Accuracy", label1='train', label2='test'):
    # 플롯 크기 설정
    plt.figure(figsize=(10, 8))

    # 1번째 서브플롯: Train_loss vs test
    plt.subplot(2, 1, 1)
    x = np.arange(len(train_accs))
    plt.plot(x, train_accs, label='train', color='blue')  # 훈련 손실
    plt.plot(x, test_accs, label='val', color='red', marker='o')  # 검증 손실
    plt.title("Train_loss vs test_loss")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend()


    # 플롯 간격 조정 및 출력
    plt.tight_layout()
    plt.show()

In [54]:
def accuracy(model, dataloader):
  cnt = 0
  acc = 0

  for data in dataloader:
    inputs, labels = data
    inputs, labels = inputs.to(device), labels.to(device)

    preds = model(inputs)
    preds = torch.argmax(preds, dim=-1)
    #preds = (preds > 0).long()[..., 0]

    cnt += labels.shape[0]
    acc += (labels == preds).sum().item()

  return acc / cnt

In [ ]:
from torch.optim import Adam
from tqdm import tqdm

lr = 0.001
model = model.to(device)
loss_fn = nn.CrossEntropyLoss()

optimizer = Adam(model.parameters(), lr=lr)
n_epochs = 10

train_loss = []
test_loss = []

for epoch in range(n_epochs):
  total_loss = 0.
  model.train()
  for data in tqdm(train_loader):
    model.zero_grad()
    inputs, attention_mask, labels = data
    inputs, attention_mask, labels = inputs.to(device), attention_mask.to(device), labels.to(device)

    preds = model(inputs, attention_mask)
    loss = loss_fn(preds, labels)
    loss.backward()
    optimizer.step()

    total_loss += loss.item()

  with torch.no_grad():
    model.eval()
    train_acc = accuracy(model, train_loader)
    test_acc = accuracy(model, test_loader)
    train_loss.append(train_acc)
    test_loss.append(test_acc)
    live_plot(train_loss, test_loss)
    print(f"=========> Train acc: {train_acc:.3f} | Test acc: {test_acc:.3f}")

  0%|          | 0/1875 [00:00<?, ?it/s]/opt/anaconda3/envs/myenv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:2847: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
  1%|          | 10/1875 [02:27<7:50:48, 15.15s/it]

Loss가 잘 떨어지고, 이전에 우리가 구현한 Transformer보다 더 빨리 수렴하는 것을 알 수 있습니다.